In [441]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import graphviz as gr
from linearmodels.iv import IV2SLS

%matplotlib inline

pd.set_option("display.max_columns", 5)
style.use("fivethirtyeight")

In [442]:
data = pd.read_csv("../Casual-Inference/data/income_data/train.csv")
data = data.dropna()
data.describe()
data

,age,workclass,...,native-country,income_>50K
0,67,Private,...,United-States,1
1,17,Private,...,United-States,0
2,31,Private,...,United-States,1
3,58,State-gov,...,United-States,0
4,25,State-gov,...,United-States,0
...,...,...,...,...,...
43952,52,Private,...,United-States,1
43953,19,Private,...,United-States,0
43954,30,Private,...,United-States,0
43955,46,Private,...,United-States,0


In [443]:
data = data.dropna(axis = 0)
data = data.loc[:, ["race", "educational-num", "age", "income_>50K", "occupation", "relationship", "marital-status", "native-country", "gender", "capital-gain", "capital-loss", "hours-per-week", "workclass"]]
data.rename(columns = {'educational-num':'educational_num', "income_>50K": "income_bigger_than_50K", 'marital-status':'marital_status'}, inplace = True)
data["race"] = data["race"].replace(to_replace ="Amer-Indian-Eskimo",
                 value ="Indian")
data["race"] = data["race"].replace(to_replace ="Asian-Pac-Islander",
                 value ="Asian")
occupationDict = {
    "Exec-managerial" : 0,
    "Other-service" : 1,
    "Transport-moving" : 2,
    "Adm-clerical" : 3,
    "Machine-op-inspct" : 4,
    "Sales" : 5,
    "Handlers-cleaners" : 6,
    "Farming-fishing" : 7,
    "Protective-serv" : 8,
    "Prof-specialty" : 9,
    "Craft-repair" : 10,
    "Tech-support" : 11,
    "Priv-house-serv" : 12,
    "Armed-Forces" : 13
}
data["occupation"] = data["occupation"].map(occupationDict)

In [444]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils.np_utils import to_categorical
from tensorflow import keras

In [445]:
X = data[['educational_num', 'age', 'race', 'gender', 'marital_status', 'native-country', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass']]
X = pd.get_dummies(X)
X

,educational_num,age,...,workclass_State-gov,workclass_Without-pay
0,16,67,...,0,0
1,8,17,...,0,0
2,13,31,...,0,0
3,4,58,...,1,0
4,10,25,...,1,0
...,...,...,...,...,...
43952,13,52,...,0,0
43953,9,19,...,0,0
43954,10,30,...,0,0
43955,13,46,...,0,0


In [446]:
y = data['occupation']
y

0        0
1        1
2        0
3        2
4        1
        ..
43952    0
43953    1
43954    5
43955    5
43956    6
Name: occupation, Length: 40727, dtype: int64

In [447]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.15)

In [448]:
encoder=LabelEncoder()
y_train_encoded=encoder.fit_transform(y_train)
y_val_encoded=encoder.fit_transform(y_val)
y_test_encoded=encoder.fit_transform(y_test)

y_train_cat=to_categorical(y_train_encoded)
y_val_cat=to_categorical(y_val_encoded)
y_test_cat=to_categorical(y_test_encoded)


In [449]:
network = Sequential()
network.add(Dense(units=128, activation='relu'))
network.add(Dense(units=128, activation='relu'))
network.add(Dense(units=64, activation='relu'))
network.add(Dense(units=14, activation='softmax'))
opt=keras.optimizers.Adam(learning_rate=0.01,decay=0.0005)
network.compile(
  optimizer = opt,
  loss = 'categorical_crossentropy', 
  metrics = ['accuracy']
)

network.fit(X_train,y_train_cat,validation_data=(X_val,y_val_cat),epochs=200)
test_loss, test_acc = network.evaluate(X_test,y_test_cat)

print('The loss function on our test set: ' + str(test_loss))
print('Accuracy on our test set: ' + str(test_acc))

Epoch 1/200
974/974 [==============================] - 2s 2ms/step - loss: 8.4279 - accuracy: 0.2328 - val_loss: 2.0751 - val_accuracy: 0.2719
Epoch 2/200
974/974 [==============================] - 1s 1ms/step - loss: 2.0368 - accuracy: 0.2889 - val_loss: 2.0057 - val_accuracy: 0.2890
Epoch 3/200
974/974 [==============================] - 1s 1ms/step - loss: 1.9906 - accuracy: 0.3039 - val_loss: 1.9849 - val_accuracy: 0.3119
Epoch 4/200
974/974 [==============================] - 1s 1ms/step - loss: 1.9716 - accuracy: 0.3095 - val_loss: 1.9655 - val_accuracy: 0.3051
Epoch 5/200
974/974 [==============================] - 1s 1ms/step - loss: 1.9602 - accuracy: 0.3125 - val_loss: 1.9911 - val_accuracy: 0.2908
Epoch 6/200
974/974 [==============================] - 1s 1ms/step - loss: 1.9523 - accuracy: 0.3165 - val_loss: 1.9435 - val_accuracy: 0.3035
Epoch 7/200
974/974 [==============================] - 1s 1ms/step - loss: 1.9462 - accuracy: 0.3143 - val_loss: 1.9441 - val_accuracy: 0.3171

KeyboardInterrupt: 